In [1]:
import numpy as np
import pandas as pd
import random
import sklearn

In [2]:
import sys
sys.path.append('..')


In [3]:
from feature_extract import *
from utils import load_data
from match_func import *
from models.lgb_model import lgb_model_for_upload,lgb_model_for_offline_test

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\18707\AppData\Local\Temp\jieba.cache
Loading model cost 0.703 seconds.
Prefix dict has been built successfully.


In [4]:
import os
path0=os.path.abspath(os.path.dirname(os.getcwd()))
path = path0 +'/data/info_dataset/'
path1 = path0+'/data/'

In [5]:
# get feature

In [6]:
person =person_feature(path)
intent_table = person_intent(path)
work_history, work_history_table = work_history(path)
cert_table = certificate(path)
project_history, project_history_table = project_history(path)
job_table = job_table(path)

In [98]:
# load data
train_data, valid_data, test_data =load_data(path1)
del train_data['Unnamed: 0']
del valid_data['Unnamed: 0']
del test_data['Unnamed: 0']

In [8]:
data_all = pd.concat([train_data,valid_data,test_data], ignore_index=True)
candidate_data, job_data = diliver_current(data_all)

## offline

In [9]:
train=get_k_fold_data(train_data,person,intent_table,job_table,project_history_table,work_history_table,candidate_data,job_data)
valid_data0 =data_merge(valid_data,person,intent_table,job_table,project_history_table,work_history_table,candidate_data,job_data)
valid = get_data(valid_data0, train)
test_data0 =data_merge(test_data,person,intent_table,job_table,project_history_table,work_history_table,candidate_data,job_data)
test= get_data(test_data0, train)

In [10]:
train.shape,valid.shape,test.shape

((28232, 57), (7059, 57), (70774, 57))

In [11]:
temp_data= pd.concat([train,valid, test], ignore_index=True)

In [12]:
def deliver_record_feature(data_feature):
    for f1, f2 in tqdm([["岗位编号", "求职意向岗位类别"], ["岗位编号", "应聘者专业"], ["岗位编号", "求职意向工作地点"]]):
        data_feature[f"{f1}_{f2}_nunique"] = data_feature.groupby(
            [f1])[f2].transform("nunique")
    # 最大值、最小值、均值、标准差、众数、众数占比
    for f1, f2 in tqdm([["岗位编号", "年龄"], ["岗位编号", "工作年限"], ["岗位编号", "工作经历数"], ["岗位编号", "项目经验数"], ["岗位编号", "总主要业绩字数"],
                        ["岗位编号", "最高学历"], ["岗位编号", "可到职天数"]]):
        df_temp1 = data_feature.groupby(f1)[f2].agg(
            [(f"{f1}_{f2}_mean", "mean"),
             (f"{f1}_{f2}_max", "max"),
             (f"{f1}_{f2}_min", "min"),
             (f"{f1}_{f2}_std", "std"),
             (f"{f1}_{f2}_median", np.median),
             (f"{f1}_{f2}_mode", mode),
             (f"{f1}_{f2}_mode_rate", mode_rate)]
        ).reset_index()
        data_feature = data_feature.merge(df_temp1, how="left")

    # 男性占比、女性个数占比、工作地点符合否占比。
    for f1, f2 in [["岗位编号", "性别"], ["岗位编号", "工作地点符合否"]]:  # 性别众数及占比，工作地点符合及占比
        df_temp2 = data_feature.groupby(f1)[f2].agg(
            [
                (f"{f1}_{f2}_mode", mode),
                (f"{f1}_{f2}_mode_rate", mode_rate)]
        ).reset_index()
        data_feature = data_feature.merge(df_temp2, how="left")
    return data_feature

In [13]:
temp_data=deliver_record_feature(temp_data)

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:09<00:00,  1.33s/it]


In [20]:
pd.DataFrame(list(temp_data.columns)).to_csv(r'columns.csv')

In [14]:
train.shape,valid.shape,test.shape,temp_data.shape

((28232, 57), (7059, 57), (70774, 57), (106065, 113))

In [373]:
temp_data.head()

,岗位编号,求职者编号,标签,性别,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,最近所在行业,...,岗位编号_可到职天数_max,岗位编号_可到职天数_min,岗位编号_可到职天数_std,岗位编号_可到职天数_median,岗位编号_可到职天数_mode,岗位编号_可到职天数_mode_rate,岗位编号_性别_mode,岗位编号_性别_mode_rate,岗位编号_工作地点符合否_mode,岗位编号_工作地点符合否_mode_rate
0,775040,5969825,0,1,12,3.0,None,34,品质/质检,NaN,...,180.0,7.0,19.058528,7.0,7.0,0.872093,1,0.738636,1.0,0.886364
1,799873,6139430,0,1,12,4.0,机电一体化,37,机械设计,零售百货行业,...,180.0,7.0,21.468561,7.0,7.0,0.754717,1,0.957944,1.0,0.957944
2,779698,6083324,0,1,0,1.0,特殊教育学,31,仓储管理,NaN,...,180.0,7.0,12.886417,7.0,7.0,0.935043,1,0.856661,0.0,0.984823
3,771109,5873085,0,0,6,4.0,财务管理,33,NaN,NaN,...,180.0,7.0,11.683337,7.0,7.0,0.892593,0,0.912088,0.0,0.948718
4,823833,6218282,0,0,0,3.0,英语,29,客户服务,NaN,...,180.0,7.0,14.694844,7.0,7.0,0.903654,0,0.891089,0.0,0.980198


## 添加求职者-岗位的匹配信息

In [190]:
data_all

,岗位编号,求职者编号,标签
0,775040,5969825,0
1,815337,6212884,0
2,770107,5451198,0
3,43795661,320873919,1
4,798922,5650936,0
...,...,...,...
106060,44350657,321173912,0
106061,44350657,321377962,0
106062,47578782,319583916,0
106063,47649893,6079047,0


In [111]:
data_all.head()
data_all.shape

(106065, 3)

In [112]:
data_all_tmp = data_all.merge(intent_table, on="求职者编号", how="left")
data_all_tmp = data_all_tmp.merge(job_data, on="岗位编号", how="left")

In [113]:
data_all_tmp = data_all_tmp.merge(person, on="求职者编号", how="left")
data_all_tmp = data_all_tmp.merge(job_table, on="岗位编号", how="left")

In [114]:
for func in ["simhash", "sim_jaccard", "difflibs"]:
        for ud in ["应聘者专业", "最近工作岗位", "最近所在行业", "专业特长",
                   "求职意向岗位类别", "求职意向所在行业", "自荐信"]:
            for jd in ["招聘职位", "对应聘者的专业要求", "具体要求"]:
                st = t.time()
                data_all_tmp[f"{ud}_{jd}_{func}"] = data_all_tmp.apply(lambda x: eval(func)(x[ud], x[jd]), axis=1)
                print(f"{ud}_{jd}_{func}，共耗时:", t.time() - st)

应聘者专业_招聘职位_simhash，共耗时: 42.26428723335266
应聘者专业_对应聘者的专业要求_simhash，共耗时: 51.95408082008362
应聘者专业_具体要求_simhash，共耗时: 89.82261204719543
最近工作岗位_招聘职位_simhash，共耗时: 46.14970636367798
最近工作岗位_对应聘者的专业要求_simhash，共耗时: 42.82719445228577
最近工作岗位_具体要求_simhash，共耗时: 87.93634223937988
最近所在行业_招聘职位_simhash，共耗时: 43.30438733100891
最近所在行业_对应聘者的专业要求_simhash，共耗时: 47.43039274215698
最近所在行业_具体要求_simhash，共耗时: 84.6497802734375
专业特长_招聘职位_simhash，共耗时: 83.13723707199097
专业特长_对应聘者的专业要求_simhash，共耗时: 72.54772758483887
专业特长_具体要求_simhash，共耗时: 106.45567846298218
求职意向岗位类别_招聘职位_simhash，共耗时: 44.421353578567505
求职意向岗位类别_对应聘者的专业要求_simhash，共耗时: 44.61356472969055
求职意向岗位类别_具体要求_simhash，共耗时: 78.405437707901
求职意向所在行业_招聘职位_simhash，共耗时: 42.73877763748169
求职意向所在行业_对应聘者的专业要求_simhash，共耗时: 45.361088514328
求职意向所在行业_具体要求_simhash，共耗时: 79.20682501792908
自荐信_招聘职位_simhash，共耗时: 81.16894745826721
自荐信_对应聘者的专业要求_simhash，共耗时: 84.36542558670044
自荐信_具体要求_simhash，共耗时: 114.31796526908875
应聘者专业_招聘职位_sim_jaccard，共耗时: 21.263596057891846
应聘者专业_对应聘者的专业要求_sim_jac

In [429]:
data_all_tmp[data_all_tmp.columns[-64:]].drop_duplicates().求职者编号_岗位编号.value_counts()

3513200802838        1
490915343668705      1
5826377775090        1
32064790043795661    1
5990345762697        1
                    ..
5393746805640        1
6031185770860        1
6232067773644        1
32144191843892657    1
5981094782877        1
Name: 求职者编号_岗位编号, Length: 106065, dtype: int64

In [209]:
data_all_tmp["求职者编号_岗位编号"]=data_all_tmp["求职者编号"].astype("str")+data_all_tmp["岗位编号"].astype("str")

In [445]:
data_all_tmp[data_all_tmp.求职者编号_岗位编号=="5873085771109"]["求职意向岗位类别_对应聘者的专业要求_simhash"]

46    0.484848
Name: 求职意向岗位类别_对应聘者的专业要求_simhash, dtype: float64

In [411]:
temp_data["求职者编号_岗位编号"]=temp_data["求职者编号"].astype("str")+temp_data["岗位编号"].astype("str")

In [447]:
temp_data.head(1)

,岗位编号,求职者编号,标签,求职者编号_岗位编号,性别,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,...,专业特长_具体要求_difflibs,求职意向岗位类别_招聘职位_difflibs,求职意向岗位类别_对应聘者的专业要求_difflibs,求职意向岗位类别_具体要求_difflibs,求职意向所在行业_招聘职位_difflibs,求职意向所在行业_对应聘者的专业要求_difflibs,求职意向所在行业_具体要求_difflibs,自荐信_招聘职位_difflibs,自荐信_对应聘者的专业要求_difflibs,自荐信_具体要求_difflibs
0,775040,5969825,0,5969825775040,1,12,3.0,0,34,281,...,0.18543,0.0,0.0,0.12,0.0,0.0,0.0,0.0,0.0,0.104167


In [412]:
temp_data = temp_data[["岗位编号", "求职者编号", "标签","求职者编号_岗位编号"] + [column for column in temp_data.columns if
                                                   column not in ["岗位编号", "求职者编号", "标签","求职者编号_岗位编号"]]]

In [432]:
temp_data.shape

(106065, 177)

In [409]:
data_all_tmp[data_all_tmp.columns[-64:]].drop_duplicates()

,应聘者专业_招聘职位_simhash,应聘者专业_对应聘者的专业要求_simhash,应聘者专业_具体要求_simhash,最近工作岗位_招聘职位_simhash,最近工作岗位_对应聘者的专业要求_simhash,最近工作岗位_具体要求_simhash,最近所在行业_招聘职位_simhash,最近所在行业_对应聘者的专业要求_simhash,最近所在行业_具体要求_simhash,专业特长_招聘职位_simhash,...,求职意向岗位类别_招聘职位_difflibs,求职意向岗位类别_对应聘者的专业要求_difflibs,求职意向岗位类别_具体要求_difflibs,求职意向所在行业_招聘职位_difflibs,求职意向所在行业_对应聘者的专业要求_difflibs,求职意向所在行业_具体要求_difflibs,自荐信_招聘职位_difflibs,自荐信_对应聘者的专业要求_difflibs,自荐信_具体要求_difflibs,求职者编号_岗位编号
0,0.530303,1.000000,0.454545,0.560606,0.515152,0.507692,0.530303,1.000000,0.454545,0.636364,...,0.000000,0.000000,0.120000,0.0,0.0,0.000000,0.000000,0.000000,0.104167,5969825775040
4,0.469697,0.560606,0.545455,0.606061,0.484848,0.530303,0.575758,0.545455,0.500000,0.484848,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.260163,6212884815337
8,0.666667,0.545455,0.507937,0.424242,0.515152,0.523077,0.500000,0.469697,0.468750,0.545455,...,0.444444,0.500000,0.148148,0.0,0.5,0.148148,0.000000,0.000000,0.000000,5451198770107
11,0.461538,0.621212,0.555556,0.424242,0.575758,0.560606,0.484848,0.515152,0.500000,0.538462,...,0.333333,0.000000,0.088235,0.0,0.0,0.029851,0.000000,0.000000,0.213115,32087391943795661
12,0.515152,0.545455,0.575758,0.530303,0.461538,0.439394,0.500000,0.500000,0.560606,0.439394,...,0.153846,0.235294,0.000000,0.0,0.0,0.000000,0.021978,0.042105,0.058824,5650936798922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250039,0.469697,0.454545,0.515152,0.545455,0.530303,0.439394,0.606061,0.560606,0.469697,0.530303,...,0.266667,0.000000,0.048193,0.0,0.0,0.000000,0.083333,0.000000,0.159574,32117391244350657
250040,0.484848,0.590909,0.560606,0.560606,1.000000,0.515152,0.560606,1.000000,0.515152,0.621212,...,0.166667,0.000000,0.086957,0.0,0.0,0.000000,0.024540,0.000000,0.199134,32137796244350657
250041,0.476923,0.560606,0.515152,0.500000,1.000000,0.469697,0.500000,1.000000,0.469697,0.615385,...,0.000000,0.000000,0.000000,0.0,1.0,0.000000,0.000000,0.000000,0.186047,31958391647578782
250042,0.590909,1.000000,0.409091,0.454545,0.469697,0.437500,0.515152,0.469697,0.507692,0.424242,...,0.000000,0.000000,0.031746,0.0,1.0,0.000000,0.000000,0.000000,0.200957,607904747649893


In [413]:
temp_data = pd.merge(temp_data,data_all_tmp[data_all_tmp.columns[-64:]].drop_duplicates(),on='求职者编号_岗位编号')

In [448]:
temp_data.head()

,岗位编号,求职者编号,标签,求职者编号_岗位编号,性别,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,...,专业特长_具体要求_difflibs,求职意向岗位类别_招聘职位_difflibs,求职意向岗位类别_对应聘者的专业要求_difflibs,求职意向岗位类别_具体要求_difflibs,求职意向所在行业_招聘职位_difflibs,求职意向所在行业_对应聘者的专业要求_difflibs,求职意向所在行业_具体要求_difflibs,自荐信_招聘职位_difflibs,自荐信_对应聘者的专业要求_difflibs,自荐信_具体要求_difflibs
0,775040,5969825,0,5969825775040,1,12,3.0,0,34,281,...,0.185430,0.000000,0.000000,0.120000,0.0,0.0,0.000000,0.000000,0.000000,0.104167
1,799873,6139430,0,6139430799873,1,12,4.0,216,37,86,...,0.062016,0.181818,0.181818,0.019417,0.0,0.0,0.000000,0.025806,0.025806,0.202429
2,779698,6083324,0,6083324779698,1,0,1.0,276,31,235,...,0.000000,0.285714,0.166667,0.026667,0.0,0.0,0.000000,0.000000,0.000000,0.191176
3,771109,5873085,0,5873085771109,0,6,4.0,373,33,226,...,0.000000,0.500000,0.000000,0.097561,0.0,0.0,0.000000,0.093023,0.000000,0.421053
4,823833,6218282,0,6218282823833,0,0,3.0,349,29,12,...,0.155844,0.000000,0.000000,0.000000,0.0,0.0,0.027778,0.000000,1.000000,0.000000


In [443]:
train.head().to_csv(r'result.csv')

In [381]:
data_all_feature.head(1)

,岗位编号,求职者编号,标签,求职者编号_岗位编号,性别,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,...,专业特长_具体要求_difflibs,求职意向岗位类别_招聘职位_difflibs,求职意向岗位类别_对应聘者的专业要求_difflibs,求职意向岗位类别_具体要求_difflibs,求职意向所在行业_招聘职位_difflibs,求职意向所在行业_对应聘者的专业要求_difflibs,求职意向所在行业_具体要求_difflibs,自荐信_招聘职位_difflibs,自荐信_对应聘者的专业要求_difflibs,自荐信_具体要求_difflibs
0,775040,5969825,0,5969825775040,1,12,3.0,None,34,品质/质检,...,0.18543,0.0,0.0,0.12,0.0,0.0,0.0,0.0,0.0,0.104167


## 添加求职者历史信息

In [202]:
data_all_tmp = data_all_tmp.merge(work_history, on="求职者编号", how="left")
data_all_tmp = data_all_tmp.merge(project_history, on="求职者编号", how="left")

In [ ]:
    
for func in ["simhash", "sim_jaccard", "difflibs"]:
    for wh in ["工作经历岗位类别", "工作经历单位所属行业", "工作经历主要业绩"]:
        for jd in ["招聘职位", "对应聘者的专业要求", "具体要求"]:
            st = t.time()
            data_all_tmp[f"{wh}_{jd}_{func}"] = data_all_tmp.apply(lambda x: eval(func)(x[wh], x[jd]),
                                                             axis=1)
            df_temp1 = data_all_tmp.groupby("求职者编号_岗位编号")[f"{wh}_{jd}_{func}"].agg(
                [(f"{wh}_{jd}_{func}_mean", "mean"),
                 (f"{wh}_{jd}_{func}_max", "max"),
                 (f"{wh}_{jd}_{func}_min", "min"),
                 (f"{wh}_{jd}_{func}_std", "std")]
            ).reset_index()
            data_all_tmp = data_all_tmp.merge(df_temp1, on="求职者编号_岗位编号", how="left")
            print(f"{wh}_{jd}_{func}，共耗时:", t.time() - st)
"""project history column:["项目名称", "项目说明", "职责说明", "关键技术"]"""
for func in ["simhash", "sim_jaccard", "difflibs"]:
    for ph in ["项目名称", "项目说明", "职责说明", "关键技术"]:
        for jd in ["招聘职位", "对应聘者的专业要求", "具体要求"]:
            st = t.time()
            data_temp2[f"{ph}_{jd}_{func}"] = data_temp2.apply(lambda x: eval(func)(x[ph], x[jd]),
                                                               axis=1)
            df_temp2 = data_temp2.groupby("求职者编号_岗位编号")[f"{ph}_{jd}_{func}"].agg(
                [(f"{ph}_{jd}_{func}_mean", "mean"),
                 (f"{ph}_{jd}_{func}_max", "max"),
                 (f"{ph}_{jd}_{func}_min", "min"),
                 (f"{ph}_{jd}_{func}_std", "std")]
            ).reset_index()
            data = data.merge(df_temp2, on="求职者编号_岗位编号", how="left")
            print(f"{ph}_{jd}_{func}，共耗时:", t.time() - st)

## 划分数据集

In [15]:
from sklearn.preprocessing import LabelEncoder

for f in temp_data.select_dtypes('object'):
    le = LabelEncoder()
    print(f)
    temp_data[f] = le.fit_transform(temp_data[f].astype('str'))
temp_data["招聘对象"] =le.fit_transform(temp_data["招聘对象"].astype('str'))
temp_data["最近工作岗位"] =le.fit_transform(temp_data["最近工作岗位"].astype('str'))
temp_data["求职意向岗位类别"] =le.fit_transform(temp_data["求职意向岗位类别"].astype('str'))
# none one-hot
data_all_feature=temp_data

应聘者专业
最近工作岗位
最近所在行业
当前工作所在地
语言能力
专业特长
招聘职位
对应聘者的专业要求
岗位工作地点
具体要求
自荐信
求职意向工作地点
求职意向所在行业
其他说明
汉语语言能力
英语语言能力
粤语语言能力
日语语言能力


In [16]:
temp_data

,岗位编号,求职者编号,标签,性别,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,最近所在行业,...,岗位编号_可到职天数_max,岗位编号_可到职天数_min,岗位编号_可到职天数_std,岗位编号_可到职天数_median,岗位编号_可到职天数_mode,岗位编号_可到职天数_mode_rate,岗位编号_性别_mode,岗位编号_性别_mode_rate,岗位编号_工作地点符合否_mode,岗位编号_工作地点符合否_mode_rate
0,775040,5969825,0,1,12,3.0,0,34,281,0,...,180.0,7.0,19.058528,7.0,7.0,0.872093,1,0.738636,1.0,0.886364
1,799873,6139430,0,1,12,4.0,216,37,86,116,...,180.0,7.0,21.468561,7.0,7.0,0.754717,1,0.957944,1.0,0.957944
2,779698,6083324,0,1,0,1.0,276,31,235,0,...,180.0,7.0,12.886417,7.0,7.0,0.935043,1,0.856661,0.0,0.984823
3,771109,5873085,0,0,6,4.0,373,33,226,0,...,180.0,7.0,11.683337,7.0,7.0,0.892593,0,0.912088,0.0,0.948718
4,823833,6218282,0,0,0,3.0,349,29,12,0,...,180.0,7.0,14.694844,7.0,7.0,0.903654,0,0.891089,0.0,0.980198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106060,44350657,321173912,0,1,5,4.0,377,26,198,38,...,180.0,7.0,56.303740,7.0,7.0,0.555556,1,0.888889,1.0,1.000000
106061,44350657,321377962,0,1,0,4.0,292,28,226,0,...,180.0,7.0,56.303740,7.0,7.0,0.555556,1,0.888889,1.0,1.000000
106062,47578782,319583916,0,0,0,4.0,302,28,226,0,...,7.0,7.0,NaN,7.0,7.0,1.000000,0,1.000000,0.0,1.000000
106063,47649893,6079047,0,1,7,4.0,0,29,174,82,...,7.0,7.0,0.000000,7.0,7.0,1.000000,1,1.000000,0.0,1.000000


In [17]:
data_all_feature.head()

,岗位编号,求职者编号,标签,性别,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,最近所在行业,...,岗位编号_可到职天数_max,岗位编号_可到职天数_min,岗位编号_可到职天数_std,岗位编号_可到职天数_median,岗位编号_可到职天数_mode,岗位编号_可到职天数_mode_rate,岗位编号_性别_mode,岗位编号_性别_mode_rate,岗位编号_工作地点符合否_mode,岗位编号_工作地点符合否_mode_rate
0,775040,5969825,0,1,12,3.0,0,34,281,0,...,180.0,7.0,19.058528,7.0,7.0,0.872093,1,0.738636,1.0,0.886364
1,799873,6139430,0,1,12,4.0,216,37,86,116,...,180.0,7.0,21.468561,7.0,7.0,0.754717,1,0.957944,1.0,0.957944
2,779698,6083324,0,1,0,1.0,276,31,235,0,...,180.0,7.0,12.886417,7.0,7.0,0.935043,1,0.856661,0.0,0.984823
3,771109,5873085,0,0,6,4.0,373,33,226,0,...,180.0,7.0,11.683337,7.0,7.0,0.892593,0,0.912088,0.0,0.948718
4,823833,6218282,0,0,0,3.0,349,29,12,0,...,180.0,7.0,14.694844,7.0,7.0,0.903654,0,0.891089,0.0,0.980198


In [18]:
train_data.shape,valid.shape,train.shape

((28232, 3), (7059, 57), (28232, 57))

In [19]:
train0 = data_all_feature[:train_data.shape[0]]#.标签.value_counts()
valid0 = data_all_feature[train_data.shape[0]:train_data.shape[0]+valid_data.shape[0]]
train_label = train0[["标签"]]
valid_label = valid0[["标签"]]
train=train0[list(train0.columns)[:2]+list(train0.columns)[3:]]
valid=valid0[list(train0.columns)[:2]+list(train0.columns)[3:]]


# k-fold offline 分析特征的有效性

In [20]:
import lightgbm
from sklearn.metrics import f1_score
params = {"objective": "binary",
          "learning_rate": 0.05,
          "max_depth": 6,
          "num_leaves": 32,
          "verbose": -1,
          "bagging_fraction": 0.8,
          "feature_fraction": 0.9,
          'subsample':0.85,
          'bagging_freq':1,
          'random_state':2048}
def lgb_model_for_offline_test(train, train_label, valid, valid_label):
    train_set = lightgbm.Dataset(train.values, label=train_label)
    val_set = lightgbm.Dataset(valid.values, label=valid_label)
    
    model = lightgbm.train(train_set=train_set,
                           valid_sets=[val_set],
                           num_boost_round=3000,
                           params=params,
                           feval=lgb_f1_score,
                           early_stopping_rounds=100)
    return model
def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat)  # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True

In [21]:
temp_data.head()

,岗位编号,求职者编号,标签,性别,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,最近所在行业,...,岗位编号_可到职天数_max,岗位编号_可到职天数_min,岗位编号_可到职天数_std,岗位编号_可到职天数_median,岗位编号_可到职天数_mode,岗位编号_可到职天数_mode_rate,岗位编号_性别_mode,岗位编号_性别_mode_rate,岗位编号_工作地点符合否_mode,岗位编号_工作地点符合否_mode_rate
0,775040,5969825,0,1,12,3.0,0,34,281,0,...,180.0,7.0,19.058528,7.0,7.0,0.872093,1,0.738636,1.0,0.886364
1,799873,6139430,0,1,12,4.0,216,37,86,116,...,180.0,7.0,21.468561,7.0,7.0,0.754717,1,0.957944,1.0,0.957944
2,779698,6083324,0,1,0,1.0,276,31,235,0,...,180.0,7.0,12.886417,7.0,7.0,0.935043,1,0.856661,0.0,0.984823
3,771109,5873085,0,0,6,4.0,373,33,226,0,...,180.0,7.0,11.683337,7.0,7.0,0.892593,0,0.912088,0.0,0.948718
4,823833,6218282,0,0,0,3.0,349,29,12,0,...,180.0,7.0,14.694844,7.0,7.0,0.903654,0,0.891089,0.0,0.980198


In [22]:
temp_data[temp_data.求职者编号_岗位编号==37815]["求职意向岗位类别_对应聘者的专业要求_simhash"]

AttributeError: 'DataFrame' object has no attribute '求职者编号_岗位编号'

In [23]:
import lightgbm
from sklearn.model_selection import KFold
X = pd.concat([train,valid],ignore_index=True).reset_index(drop=True)
y = pd.concat([train_label,valid_label],ignore_index=True).reset_index(drop=True)
X =X[X.columns[:112]]
oof = []
df_importance_list = []
# model = lightgbm.LGBMClassifier(num_leaves=64,
#                            max_depth=10,
#                            learning_rate=0.1,
#                            n_estimators=1000000,
#                            subsample=0.8,
#                            feature_fraction=0.8,
#                            reg_alpha=0.5,
#                            reg_lambda=0.5,
#                            random_state=2048,
#                            metric='auc')
kfold = KFold(n_splits=5, random_state=2048, shuffle=True)
for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(X, y)):
    X_train = X.loc[trn_idx]
    Y_train = y.loc[trn_idx]

    X_val = X.loc[val_idx]
    Y_val = y.loc[val_idx]
    train_set = lightgbm.Dataset(X_train.values, label=Y_train)
    val_set = lightgbm.Dataset(X_val.values, label=Y_val)
    
    lgb_model = lightgbm.train(train_set=train_set,
                           valid_sets=[val_set],
                           num_boost_round=3000,
                           params=params,
                           feval=lgb_f1_score,
                           early_stopping_rounds=100)
    print('\nFold_{} Training ================================\n'.format(fold_id + 1))

    pred_val = lgb_model.predict(X_val,num_iteration=lgb_model.best_iteration)
    df_oof = X.iloc[val_idx][['岗位编号', '求职者编号']].copy()
    df_oof['标签'] = y.iloc[val_idx][['标签']].copy()
    df_oof['pred'] = pred_val
    oof.append(df_oof)


    df_importance = pd.DataFrame({
        'column': list(X.columns),
        'importance': lgb_model.feature_importance(),
    })
    df_importance_list.append(df_importance)


[1]	valid_0's binary_logloss: 0.409228	valid_0's f1: 0
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's binary_logloss: 0.380464	valid_0's f1: 0
[3]	valid_0's binary_logloss: 0.356827	valid_0's f1: 0
[4]	valid_0's binary_logloss: 0.336984	valid_0's f1: 0
[5]	valid_0's binary_logloss: 0.319861	valid_0's f1: 0
[6]	valid_0's binary_logloss: 0.304387	valid_0's f1: 0
[7]	valid_0's binary_logloss: 0.290703	valid_0's f1: 0
[8]	valid_0's binary_logloss: 0.278244	valid_0's f1: 0
[9]	valid_0's binary_logloss: 0.267223	valid_0's f1: 0
[10]	valid_0's binary_logloss: 0.257211	valid_0's f1: 0.678824
[11]	valid_0's binary_logloss: 0.25009	valid_0's f1: 0.704729
[12]	valid_0's binary_logloss: 0.241552	valid_0's f1: 0.734648
[13]	valid_0's binary_logloss: 0.23356	valid_0's f1: 0.764156
[14]	valid_0's binary_logloss: 0.226239	valid_0's f1: 0.776994
[15]	valid_0's binary_logloss: 0.219405	valid_0's f1: 0.805263
[16]	valid_0's binary_logloss: 0.213018	valid_0's f1: 0.811685
[17

[134]	valid_0's binary_logloss: 0.110543	valid_0's f1: 0.863296
[135]	valid_0's binary_logloss: 0.110492	valid_0's f1: 0.86236
[136]	valid_0's binary_logloss: 0.110395	valid_0's f1: 0.861293
[137]	valid_0's binary_logloss: 0.110412	valid_0's f1: 0.861827
[138]	valid_0's binary_logloss: 0.110355	valid_0's f1: 0.86236
[139]	valid_0's binary_logloss: 0.110311	valid_0's f1: 0.861293
[140]	valid_0's binary_logloss: 0.110296	valid_0's f1: 0.861567
[141]	valid_0's binary_logloss: 0.11021	valid_0's f1: 0.862506
[142]	valid_0's binary_logloss: 0.110127	valid_0's f1: 0.862763
[143]	valid_0's binary_logloss: 0.110223	valid_0's f1: 0.863296
[144]	valid_0's binary_logloss: 0.110196	valid_0's f1: 0.863572
[145]	valid_0's binary_logloss: 0.110228	valid_0's f1: 0.862763
[146]	valid_0's binary_logloss: 0.110306	valid_0's f1: 0.861956
[147]	valid_0's binary_logloss: 0.110369	valid_0's f1: 0.862085
[148]	valid_0's binary_logloss: 0.110387	valid_0's f1: 0.862617
[149]	valid_0's binary_logloss: 0.110431	va

[21]	valid_0's binary_logloss: 0.194331	valid_0's f1: 0.813594
[22]	valid_0's binary_logloss: 0.190124	valid_0's f1: 0.818648
[23]	valid_0's binary_logloss: 0.186275	valid_0's f1: 0.819857
[24]	valid_0's binary_logloss: 0.182608	valid_0's f1: 0.82263
[25]	valid_0's binary_logloss: 0.179082	valid_0's f1: 0.825977
[26]	valid_0's binary_logloss: 0.175887	valid_0's f1: 0.825139
[27]	valid_0's binary_logloss: 0.172659	valid_0's f1: 0.827621
[28]	valid_0's binary_logloss: 0.16982	valid_0's f1: 0.830321
[29]	valid_0's binary_logloss: 0.167258	valid_0's f1: 0.833583
[30]	valid_0's binary_logloss: 0.164712	valid_0's f1: 0.836237
[31]	valid_0's binary_logloss: 0.16235	valid_0's f1: 0.837394
[32]	valid_0's binary_logloss: 0.160083	valid_0's f1: 0.838294
[33]	valid_0's binary_logloss: 0.157962	valid_0's f1: 0.838199
[34]	valid_0's binary_logloss: 0.156021	valid_0's f1: 0.841481
[35]	valid_0's binary_logloss: 0.153974	valid_0's f1: 0.843195
[36]	valid_0's binary_logloss: 0.152173	valid_0's f1: 0.84

[152]	valid_0's binary_logloss: 0.118752	valid_0's f1: 0.858634
[153]	valid_0's binary_logloss: 0.118755	valid_0's f1: 0.857685
[154]	valid_0's binary_logloss: 0.1187	valid_0's f1: 0.858092
[155]	valid_0's binary_logloss: 0.1187	valid_0's f1: 0.857278
[156]	valid_0's binary_logloss: 0.118664	valid_0's f1: 0.85782
[157]	valid_0's binary_logloss: 0.118683	valid_0's f1: 0.859041
[158]	valid_0's binary_logloss: 0.118734	valid_0's f1: 0.858768
[159]	valid_0's binary_logloss: 0.118753	valid_0's f1: 0.859041
[160]	valid_0's binary_logloss: 0.118741	valid_0's f1: 0.858092
[161]	valid_0's binary_logloss: 0.118774	valid_0's f1: 0.858092
[162]	valid_0's binary_logloss: 0.118761	valid_0's f1: 0.858227
[163]	valid_0's binary_logloss: 0.118769	valid_0's f1: 0.858361
[164]	valid_0's binary_logloss: 0.118758	valid_0's f1: 0.85782
[165]	valid_0's binary_logloss: 0.118775	valid_0's f1: 0.857008
[166]	valid_0's binary_logloss: 0.118854	valid_0's f1: 0.857008
[167]	valid_0's binary_logloss: 0.118805	valid

[63]	valid_0's binary_logloss: 0.121581	valid_0's f1: 0.857003
[64]	valid_0's binary_logloss: 0.121136	valid_0's f1: 0.858118
[65]	valid_0's binary_logloss: 0.120735	valid_0's f1: 0.857004
[66]	valid_0's binary_logloss: 0.120309	valid_0's f1: 0.857977
[67]	valid_0's binary_logloss: 0.119886	valid_0's f1: 0.858394
[68]	valid_0's binary_logloss: 0.119373	valid_0's f1: 0.859504
[69]	valid_0's binary_logloss: 0.118942	valid_0's f1: 0.857836
[70]	valid_0's binary_logloss: 0.118586	valid_0's f1: 0.85839
[71]	valid_0's binary_logloss: 0.118255	valid_0's f1: 0.857696
[72]	valid_0's binary_logloss: 0.117931	valid_0's f1: 0.857696
[73]	valid_0's binary_logloss: 0.117606	valid_0's f1: 0.858664
[74]	valid_0's binary_logloss: 0.117151	valid_0's f1: 0.85908
[75]	valid_0's binary_logloss: 0.116865	valid_0's f1: 0.858527
[76]	valid_0's binary_logloss: 0.116518	valid_0's f1: 0.857696
[77]	valid_0's binary_logloss: 0.116273	valid_0's f1: 0.857834
[78]	valid_0's binary_logloss: 0.115912	valid_0's f1: 0.8

...


Fold_3 Training ================================

[1]	valid_0's binary_logloss: 0.410077	valid_0's f1: 0
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's binary_logloss: 0.381311	valid_0's f1: 0
[3]	valid_0's binary_logloss: 0.357999	valid_0's f1: 0
[4]	valid_0's binary_logloss: 0.338212	valid_0's f1: 0
[5]	valid_0's binary_logloss: 0.320778	valid_0's f1: 0
[6]	valid_0's binary_logloss: 0.305477	valid_0's f1: 0
[7]	valid_0's binary_logloss: 0.291723	valid_0's f1: 0
[8]	valid_0's binary_logloss: 0.279459	valid_0's f1: 0
[9]	valid_0's binary_logloss: 0.268243	valid_0's f1: 0
[10]	valid_0's binary_logloss: 0.258155	valid_0's f1: 0.688773
[11]	valid_0's binary_logloss: 0.251109	valid_0's f1: 0.709049
[12]	valid_0's binary_logloss: 0.242437	valid_0's f1: 0.726452
[13]	valid_0's binary_logloss: 0.234365	valid_0's f1: 0.741648
[14]	valid_0's binary_logloss: 0.227007	valid_0's f1: 0.760548
[15]	valid_0's binary_logloss: 0.22015	valid_0's f1: 0.780172
[16]	valid_0's

[132]	valid_0's binary_logloss: 0.109721	valid_0's f1: 0.863228
[133]	valid_0's binary_logloss: 0.109665	valid_0's f1: 0.863228
[134]	valid_0's binary_logloss: 0.109707	valid_0's f1: 0.863357
[135]	valid_0's binary_logloss: 0.109698	valid_0's f1: 0.863486
[136]	valid_0's binary_logloss: 0.109682	valid_0's f1: 0.863357
[137]	valid_0's binary_logloss: 0.109631	valid_0's f1: 0.862949
[138]	valid_0's binary_logloss: 0.109629	valid_0's f1: 0.863765
[139]	valid_0's binary_logloss: 0.10957	valid_0's f1: 0.863357
[140]	valid_0's binary_logloss: 0.109536	valid_0's f1: 0.863486
[141]	valid_0's binary_logloss: 0.109507	valid_0's f1: 0.865248
[142]	valid_0's binary_logloss: 0.109455	valid_0's f1: 0.864711
[143]	valid_0's binary_logloss: 0.109437	valid_0's f1: 0.864711
[144]	valid_0's binary_logloss: 0.109495	valid_0's f1: 0.863098
[145]	valid_0's binary_logloss: 0.109466	valid_0's f1: 0.862689
[146]	valid_0's binary_logloss: 0.109472	valid_0's f1: 0.863894
[147]	valid_0's binary_logloss: 0.109502	

[264]	valid_0's binary_logloss: 0.109469	valid_0's f1: 0.868868
[265]	valid_0's binary_logloss: 0.109481	valid_0's f1: 0.868868
[266]	valid_0's binary_logloss: 0.109507	valid_0's f1: 0.869278
[267]	valid_0's binary_logloss: 0.109521	valid_0's f1: 0.868334
[268]	valid_0's binary_logloss: 0.109545	valid_0's f1: 0.869278
[269]	valid_0's binary_logloss: 0.109546	valid_0's f1: 0.869688
[270]	valid_0's binary_logloss: 0.109507	valid_0's f1: 0.871287
[271]	valid_0's binary_logloss: 0.10948	valid_0's f1: 0.871819
[272]	valid_0's binary_logloss: 0.109455	valid_0's f1: 0.870466
[273]	valid_0's binary_logloss: 0.109447	valid_0's f1: 0.869278
[274]	valid_0's binary_logloss: 0.109463	valid_0's f1: 0.868744
[275]	valid_0's binary_logloss: 0.109545	valid_0's f1: 0.870633
[276]	valid_0's binary_logloss: 0.109541	valid_0's f1: 0.870633
[277]	valid_0's binary_logloss: 0.109507	valid_0's f1: 0.869688
[278]	valid_0's binary_logloss: 0.109481	valid_0's f1: 0.870633
[279]	valid_0's binary_logloss: 0.109484	

[68]	valid_0's binary_logloss: 0.122163	valid_0's f1: 0.855491
[69]	valid_0's binary_logloss: 0.121829	valid_0's f1: 0.856042
[70]	valid_0's binary_logloss: 0.121458	valid_0's f1: 0.855491
[71]	valid_0's binary_logloss: 0.121246	valid_0's f1: 0.856042
[72]	valid_0's binary_logloss: 0.120857	valid_0's f1: 0.856455
[73]	valid_0's binary_logloss: 0.120618	valid_0's f1: 0.857829
[74]	valid_0's binary_logloss: 0.120397	valid_0's f1: 0.858925
[75]	valid_0's binary_logloss: 0.120051	valid_0's f1: 0.858925
[76]	valid_0's binary_logloss: 0.119825	valid_0's f1: 0.857829
[77]	valid_0's binary_logloss: 0.119596	valid_0's f1: 0.85879
[78]	valid_0's binary_logloss: 0.119355	valid_0's f1: 0.859337
[79]	valid_0's binary_logloss: 0.119039	valid_0's f1: 0.859472
[80]	valid_0's binary_logloss: 0.118748	valid_0's f1: 0.859607
[81]	valid_0's binary_logloss: 0.118475	valid_0's f1: 0.85769
[82]	valid_0's binary_logloss: 0.118313	valid_0's f1: 0.85906
[83]	valid_0's binary_logloss: 0.118159	valid_0's f1: 0.85

[197]	valid_0's binary_logloss: 0.112178	valid_0's f1: 0.864839
[198]	valid_0's binary_logloss: 0.112035	valid_0's f1: 0.865121
[199]	valid_0's binary_logloss: 0.112059	valid_0's f1: 0.865121
[200]	valid_0's binary_logloss: 0.111995	valid_0's f1: 0.86553
[201]	valid_0's binary_logloss: 0.111955	valid_0's f1: 0.86553
[202]	valid_0's binary_logloss: 0.111953	valid_0's f1: 0.865121
[203]	valid_0's binary_logloss: 0.111932	valid_0's f1: 0.864711
[204]	valid_0's binary_logloss: 0.111904	valid_0's f1: 0.864711
[205]	valid_0's binary_logloss: 0.111912	valid_0's f1: 0.864583
[206]	valid_0's binary_logloss: 0.111945	valid_0's f1: 0.865121
[207]	valid_0's binary_logloss: 0.111928	valid_0's f1: 0.865248
[208]	valid_0's binary_logloss: 0.111952	valid_0's f1: 0.865121
[209]	valid_0's binary_logloss: 0.112017	valid_0's f1: 0.865121
[210]	valid_0's binary_logloss: 0.112069	valid_0's f1: 0.865121
[211]	valid_0's binary_logloss: 0.112053	valid_0's f1: 0.86553
[212]	valid_0's binary_logloss: 0.111981	va

In [505]:
X_val

,岗位编号,求职者编号,性别,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,最近所在行业,当前工作所在地,...,岗位编号_可到职天数_max,岗位编号_可到职天数_min,岗位编号_可到职天数_std,岗位编号_可到职天数_median,岗位编号_可到职天数_mode,岗位编号_可到职天数_mode_rate,岗位编号_性别_mode,岗位编号_性别_mode_rate,岗位编号_工作地点符合否_mode,岗位编号_工作地点符合否_mode_rate
8,834175,4641663,0,11,4.0,112,29,239,83,43,...,30.0,7.0,5.631350,7.0,7.0,0.873134,0,0.681481,1.0,0.881481
10,840046,6302000,0,0,3.0,329,27,73,94,43,...,180.0,7.0,26.345575,7.0,7.0,0.872093,0,0.953488,0.0,0.872093
15,830093,6165295,1,11,4.0,128,37,217,0,43,...,180.0,7.0,41.980825,7.0,7.0,0.882353,0,0.882353,0.0,0.941176
21,829293,6279126,1,18,4.0,33,41,142,0,2,...,30.0,7.0,4.600000,7.0,7.0,0.960000,1,0.730769,0.0,0.961538
25,779975,5907292,1,8,3.0,0,31,142,0,23,...,180.0,7.0,11.068181,7.0,7.0,0.898305,0,0.576159,0.0,0.956954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35278,770081,5758736,0,0,3.0,349,29,217,0,43,...,180.0,7.0,18.965588,7.0,7.0,0.846154,1,0.510638,0.0,0.989362
35281,796058,6109335,1,10,4.0,211,32,147,41,19,...,30.0,7.0,8.890418,7.0,7.0,0.782609,1,1.000000,0.0,1.000000
35283,791584,5985912,0,0,3.0,146,32,24,0,2,...,180.0,7.0,10.322548,7.0,7.0,0.903315,0,0.921409,0.0,0.948509
35286,772901,5119889,0,10,4.0,110,31,29,27,43,...,180.0,7.0,19.385633,7.0,7.0,0.893305,0,0.826963,0.0,0.969525


In [390]:
df_importance.to_csv('result.csv')

In [405]:
len(lgb_model.feature_importance()),len(list(test0.columns)[:2]+list(test0.columns)[3:])

(112, 176)

In [24]:
from sklearn.metrics import roc_auc_score
df_oof = pd.concat(oof)
df_oof.sort_values(['pred'], inplace=True, ascending=False)
df_oof.reset_index(drop=True, inplace=True)
df_oof['pred_label'] = 0
df_oof.loc[:int(0.14 * len(df_oof)), 'pred_label'] = 1
df_oof["pred_label_class"] =df_oof.pred.apply(lambda x:round(x) )
f1 = f1_score(df_oof["标签"], df_oof['pred_label_class'])
auc = roc_auc_score(df_oof["标签"], df_oof['pred_label_class'])
f1, auc

(0.864237160120846, 0.9004942585243706)

In [501]:
round(0.51)

1

In [496]:
df_oof[]

,岗位编号,求职者编号,标签,pred,pred_label
0,43922680,316802963,1,0.999971,1
1,43267862,317749941,1,0.999951,1
2,43922680,320945910,1,0.999947,1
3,44172666,321029922,1,0.999946,1
4,43524687,319520903,1,0.999943,1
...,...,...,...,...,...
35286,786572,5749793,0,0.000181,0
35287,813149,6261287,0,0.000181,0
35288,766805,6185851,0,0.000154,0
35289,820414,6258854,0,0.000149,0


In [246]:
df_importance = pd.concat(df_importance_list)
df_importance = df_importance.groupby([
    'column'
])['importance'].agg('mean')#.sort_values(ascending=False).reset_index()
df_importance.to_csv(r'result_import_177.csv')

In [185]:
# 112维特征：0.8642  ；0.8702
# 175维特征：0.8671  ：

## online

In [78]:
train_data.head()

,岗位编号,求职者编号,标签
0,775040,5969825,0
1,815337,6212884,0
2,770107,5451198,0
3,43795661,320873919,1
4,798922,5650936,0


In [80]:
# online
data=pd.concat([train_data,valid_data],ignore_index=True)
train_online=get_k_fold_data(data,person,intent_table,job_table,project_history_table,work_history_table,candidate_data,job_data)
test_data0 =data_merge(test_data,person,intent_table,job_table,project_history_table,work_history_table,candidate_data,job_data)
test_online = get_data(test_data0, train_online)

In [515]:
train_online["求职者编号_岗位编号"]=train_online["求职者编号"].astype("str")+train_online["岗位编号"].astype("str")
test_online["求职者编号_岗位编号"]=test_online["求职者编号"].astype("str")+test_online["岗位编号"].astype("str")
temp_data_online["求职者编号_岗位编号"]=temp_data_online["求职者编号"].astype("str")+temp_data_online["岗位编号"].astype("str")

In [81]:
# 获取投递记录相关指标的统计值
temp_data_online= pd.concat([train_online, test_online], ignore_index=True)
# data_all_feature=feature_process(temp_data)
temp_data_online=deliver_record_feature(temp_data_online)

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:09<00:00,  1.34s/it]


In [516]:
temp_data_online = temp_data_online[["岗位编号", "求职者编号", "标签","求职者编号_岗位编号"] + [column for column in temp_data_online.columns if
                                                   column not in ["岗位编号", "求职者编号", "标签","求职者编号_岗位编号"]]]
temp_data_online = pd.merge(temp_data_online,data_all_tmp[data_all_tmp.columns[-64:]].drop_duplicates(),on='求职者编号_岗位编号')

In [538]:
temp_data_online.head()

,岗位编号,求职者编号,标签,求职者编号_岗位编号,性别,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,...,专业特长_具体要求_difflibs,求职意向岗位类别_招聘职位_difflibs,求职意向岗位类别_对应聘者的专业要求_difflibs,求职意向岗位类别_具体要求_difflibs,求职意向所在行业_招聘职位_difflibs,求职意向所在行业_对应聘者的专业要求_difflibs,求职意向所在行业_具体要求_difflibs,自荐信_招聘职位_difflibs,自荐信_对应聘者的专业要求_difflibs,自荐信_具体要求_difflibs
0,775040,5969825,0,43073,1,12,3.0,0,34,203,...,0.185430,0.000000,0.000000,0.120000,0.0,0.0,0.000000,0.000000,0.000000,0.104167
1,799873,6139430,0,75369,1,12,4.0,216,37,283,...,0.062016,0.181818,0.181818,0.019417,0.0,0.0,0.000000,0.025806,0.025806,0.202429
2,779698,6083324,0,64742,1,0,1.0,276,31,152,...,0.000000,0.285714,0.166667,0.026667,0.0,0.0,0.000000,0.000000,0.000000,0.191176
3,771109,5873085,0,37815,0,6,4.0,373,33,142,...,0.000000,0.500000,0.000000,0.097561,0.0,0.0,0.000000,0.093023,0.000000,0.421053
4,823833,6218282,0,88752,0,0,3.0,349,29,24,...,0.155844,0.000000,0.000000,0.000000,0.0,0.0,0.027778,0.000000,1.000000,0.000000


In [541]:
temp_data_online[temp_data_online.求职者编号_岗位编号==37815]["求职意向岗位类别_对应聘者的专业要求_simhash"]

3    0.484848
Name: 求职意向岗位类别_对应聘者的专业要求_simhash, dtype: float64

In [82]:
for f in temp_data_online.select_dtypes('object'):
    le = LabelEncoder()
    print(f)
    temp_data_online[f] = le.fit_transform(temp_data_online[f].astype('str'))
temp_data_online["招聘对象"] =le.fit_transform(temp_data_online["招聘对象"].astype('str'))
temp_data_online["最近工作岗位"] =le.fit_transform(temp_data_online["最近工作岗位"].astype('str'))
temp_data_online["求职意向岗位类别"] =le.fit_transform(temp_data_online["求职意向岗位类别"].astype('str'))





应聘者专业
最近工作岗位
最近所在行业
当前工作所在地
语言能力
专业特长
招聘职位
对应聘者的专业要求
岗位工作地点
具体要求
自荐信
求职意向工作地点
求职意向所在行业
其他说明
汉语语言能力
英语语言能力
粤语语言能力
日语语言能力


In [520]:
temp_data_online.head()

,岗位编号,求职者编号,标签,求职者编号_岗位编号,性别,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,...,专业特长_具体要求_difflibs,求职意向岗位类别_招聘职位_difflibs,求职意向岗位类别_对应聘者的专业要求_difflibs,求职意向岗位类别_具体要求_difflibs,求职意向所在行业_招聘职位_difflibs,求职意向所在行业_对应聘者的专业要求_difflibs,求职意向所在行业_具体要求_difflibs,自荐信_招聘职位_difflibs,自荐信_对应聘者的专业要求_difflibs,自荐信_具体要求_difflibs
0,775040,5969825,0,43073,1,12,3.0,0,34,203,...,0.185430,0.000000,0.000000,0.120000,0.0,0.0,0.000000,0.000000,0.000000,0.104167
1,799873,6139430,0,75369,1,12,4.0,216,37,283,...,0.062016,0.181818,0.181818,0.019417,0.0,0.0,0.000000,0.025806,0.025806,0.202429
2,779698,6083324,0,64742,1,0,1.0,276,31,152,...,0.000000,0.285714,0.166667,0.026667,0.0,0.0,0.000000,0.000000,0.000000,0.191176
3,771109,5873085,0,37815,0,6,4.0,373,33,142,...,0.000000,0.500000,0.000000,0.097561,0.0,0.0,0.000000,0.093023,0.000000,0.421053
4,823833,6218282,0,88752,0,0,3.0,349,29,24,...,0.155844,0.000000,0.000000,0.000000,0.0,0.0,0.027778,0.000000,1.000000,0.000000


In [37]:
train.shape,valid.shape,test.shape,train_online.shape,test_online.shape

((28232, 112), (7059, 112), (70774, 57), (35291, 112), (70774, 112))

In [83]:
data_all_feature_online  =temp_data_online
#train,train_label,valid,valid_label
train0 = data_all_feature_online[:train_online.shape[0]]#.标签.value_counts()
test_online = data_all_feature_online[train_online.shape[0]:]
train_label_online = train0["标签"]
train_online=train0[list(train0.columns)[:2]+list(train0.columns)[3:]]
test_online=test_online[list(test_online.columns)[:2]+list(test_online.columns)[3:]]

In [59]:
train_online.shape,test_online.shape

((35291, 112), (70774, 112))

In [84]:
import lightgbm
model = lightgbm.train(train_set=lightgbm.Dataset(train_online.values, train_label_online.values),
                       num_boost_round=800,
                       params=params)



In [107]:
# from utils import submission
    
path2=path0+'/result/'
name ='online_112_feature_v2_class'
test_online["预测打分"] = model.predict(test_online.values)
test_online["预测类别"] =test_online["预测打分"].apply(lambda x:round(x) )
test_online = test_online.sort_values("预测打分", ascending=False, ignore_index=True)
test_online["预测"] = 0
test_online.loc[:int(0.15 * len(test_online)), ["预测"]] = 1

submit = test_online[["岗位编号", "求职者编号", "预测类别"]]
submit.columns = ["RECRUIT_ID", "PERSON_ID", "LABEL"]
submit.to_csv(path2 + name + ".csv", index=False)
del test_online["预测打分"]
del test_online["预测类别"]
del test_online["预测"]

In [108]:
test_online#[(test_data.岗位编号==42732392)&(test_data.求职者编号==4918222)]

,岗位编号,求职者编号,性别,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,最近所在行业,当前工作所在地,...,岗位编号_可到职天数_max,岗位编号_可到职天数_min,岗位编号_可到职天数_std,岗位编号_可到职天数_median,岗位编号_可到职天数_mode,岗位编号_可到职天数_mode_rate,岗位编号_性别_mode,岗位编号_性别_mode_rate,岗位编号_工作地点符合否_mode,岗位编号_工作地点符合否_mode_rate
0,44172666,321298925,0,5,4.0,349,29,288,80,43,...,180.0,7.0,36.052991,7.0,7.0,0.822222,0,0.869565,1.0,0.847826
1,42907680,316480900,0,5,3.0,0,28,174,82,19,...,14.0,7.0,1.527525,7.0,7.0,0.952381,0,0.857143,0.0,1.000000
2,43621663,320934905,0,7,3.0,329,5,288,80,43,...,180.0,7.0,26.428022,7.0,7.0,0.790909,1,0.636364,1.0,0.859091
3,42372664,316479909,1,6,3.0,318,28,288,80,56,...,30.0,7.0,3.946830,7.0,7.0,0.944444,1,0.960000,1.0,0.800000
4,43795661,316450912,0,9,3.0,59,33,196,55,70,...,180.0,7.0,19.234797,7.0,7.0,0.782123,0,0.989071,1.0,0.868852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70769,772337,6015851,1,0,4.0,364,30,226,0,43,...,30.0,7.0,3.252691,7.0,7.0,0.980000,1,0.960000,0.0,1.000000
70770,821777,6257090,1,0,4.0,145,30,226,0,2,...,30.0,7.0,7.628237,7.0,7.0,0.880000,1,0.920000,0.0,0.960000
70771,772338,5985485,1,8,4.0,149,31,15,0,19,...,30.0,7.0,4.130922,7.0,7.0,0.967742,0,0.727273,0.0,0.969697
70772,770533,6125754,1,7,4.0,145,31,226,0,0,...,180.0,7.0,21.781560,7.0,7.0,0.764706,1,0.929577,0.0,0.985915


In [109]:
submit

,RECRUIT_ID,PERSON_ID,LABEL
0,44172666,321298925,1
1,42907680,316480900,1
2,43621663,320934905,1
3,42372664,316479909,1
4,43795661,316450912,1
...,...,...,...
70769,772337,6015851,0
70770,821777,6257090,0
70771,772338,5985485,0
70772,770533,6125754,0


* one-hot +岗位编号	求职者编号 0.8633      ONLINE:0.8653
* no one hot +岗位编号	求职者编号 0.865602      ONLINE:0.8663
* 添加招聘岗位投递记录中的求职意向岗位，求职者专业，（年龄、工作经历数、项目经验数、可到职天数）统计指标、工作地点符合占比特征
* 0.8665 online：0.86748
* 添加k-折交叉的标签特征：offline:0.8772  online: 0.8711
* 112维特征：0.869288

In [2]:
import pandas as pd
df1 = pd.DataFrame([1,2,3,4,5,6])

In [5]:
df1[1]=[1,1,1,1,2,2]

In [6]:
df1

,0,1
0,1,1
1,2,1
2,3,1
3,4,1
4,5,2
5,6,2


In [7]:
f1=1
f2=0
df1.groupby(f1)[f2].agg(
        [(f"{f1}_{f2}_mean", "mean"),
         (f"{f1}_{f2}_max", "max"),
         (f"{f1}_{f2}_min", "min"),
         (f"{f1}_{f2}_std", "std")]
    ).reset_index()

,1,1_0_mean,1_0_max,1_0_min,1_0_std
0,1,2.5,4,1,1.290994
1,2,5.5,6,5,0.707107
